In [ ]:
# References:
# 1. http://professor.luzerna.ifc.edu.br/ricardo-antonello/wp-content/uploads/sites/8/2017/02/Livro-Introdu%C3%A7%C3%A3o-a-Vis%C3%A3o-Computacional-com-Python-e-OpenCV-1.pdf
# 2. https://docs.opencv.org/2.4/modules/refman.html

import cv2
import numpy as np
import matplotlib.pyplot as plt
import mahotas

In [ ]:
# put desired file here
fn = 'models/HelloWorld.jpg'
#fn = 'templateFilled.jpg'

image = cv2.imread(fn)


# resize image to keep coordinates
dim = (768, 1024)
image=cv2.resize(image, dim, interpolation =cv2.INTER_AREA)

# create a BW image from original to improve border detection
vid_bw = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY)[1]

# blur bw image
smooth = cv2.blur(vid_bw, (7, 7))


# Threshold for binarize smooth image
T = mahotas.thresholding.otsu(smooth)
binImg = smooth.copy()
binImg[binImg > T] = 255
binImg[binImg < 255] = 0
binImg = cv2.bitwise_not(binImg)

# Use Canny filter to detect contours
borders = cv2.Canny(binImg, 150, 150)

# Find and count image contours
# the key cv2.RETR_EXTERNAL = only external contours
contours, hierarchy = cv2.findContours(borders.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# count number of found contours
print("Number of Contours found = " + str(len(contours))) 

# display overlay found contours in original image
cv2.drawContours(image, contours, -1, (0, 255, 0), 1) 
# set a large display size
plt.figure(figsize=(30,22))
plt.imshow(image)
plt.show()

In [ ]:
# Next step: detect hexdecimal values to each line in first template column
# TODO: read others columns

# Going through every contours found in the image. 
centroids = np.zeros((len(contours),2),dtype=int)
for i in range(0,len(contours)):
    a = np.array(contours[i])
    x = int(a.mean(0)[0][0])
    y = int(a.mean(0)[0][1])
    centroids[i]=(x,y)
    
# these values are taken to crop contour search, must be changed to image-based values    
dx = 34
dz = 30
dy = 28
maxPx = centroids.max(0)[0] -dz
maxPy = centroids.max(0)[1] -dx
minPx = centroids.min(0)[0] -dz
minPy = centroids.min(0)[1] -dy

In [ ]:
# Create an array copy of centroids, measure relative position for each line 
ct = np.array(centroids,float)
ct = ct - [ct.min(0)[0],ct.min(0)[1]]
ct[:,0] = np.round((ct[:,0]*44/(maxPx))-2)
ct[:,1] = np.round((ct[:,1]*66/(maxPy))-2)

# At this point, centroids are integer numbers within the range (0,37) as follows:
# 0-7: first template column
# 10-17: second template column
# 20-27: third template co
# 30-37: fourth template co

# by this time, only first column was implemented


filter_arr = ct<[8,1000]
# get only 0-7 bits
ct = ct[filter_arr.all(1)]

# string lists with result for each row
arrayBIN = []
arrayNUM = []
arrayHEX = []
for i in range (0,64):
    filter_arr = ct[:,1] == i
    currentArray = ct[filter_arr]
    num = 0
    for j in range (0,len(currentArray)):
        num = num + np.power(2,7-currentArray[j][0])
    arrayHEX.append(str("{0:02X}".format(int(num))))
    arrayNUM.append(str("{0:2}".format(int(num))))
    arrayBIN.append(str("{0:08b}".format(int(num))))

In [ ]:
# Read image again and resize
dim = (768, 1024)
image2 = cv2.imread(fn)
image2 = cv2.resize(image2, dim, interpolation =cv2.INTER_AREA)

# Put points for each centroid
# TODO: print centroids only in croped area
for i in range(0,len(centroids)):
    image2 = cv2.circle(image2, (centroids[i][0],centroids[i][1]) ,radius=3, color=(0, 0, 255), thickness=2)

# Display decoded value for each row
# TODO: adjust position to any aspect ration or resolution
for i in range(0,64):
    cv2.putText(image2, arrayBIN[i], (15,46+int(14.8*i)), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255),fontScale=0.3)

# Display figure with points and annotations    
plt.figure(figsize=(30,22))
plt.imshow(image2)
plt.show()

# save file
cv2.imwrite("out_"+fn, image2) 

In [ ]:
# Hello World default code implemented in https://schweigi.github.io/assembler-simulator/ assembler simulator HEX code:
prog = ['1F','0F','48','65','6C','6C','6F','20','57','6F','72','6C','64','21','00','06','02','02','06','03','E8','38','18','00','32','00','32','01','06','01','00','03','00','02','05','03','00','12','02','12','03','15','01','02','27','1F','36','01','36','00','39']

# Compare decoded code with input code
listToStr = ' '.join([str(elem) for elem in prog]) 
print(listToStr)
listToStr = ' '.join([str(elem) for elem in arrayHEX]) 
print(listToStr)